# Section 0 (Preliminaries)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

import os
os.makedirs("outputs/figures", exist_ok=True)
os.makedirs("outputs/tables", exist_ok=True)

def save_output(obj, name, kind="figure"):
    """
    Save figures or tables to the outputs folder.

    Parameters:
        obj  : matplotlib figure OR pandas DataFrame
        name : str, filename without extension
        kind : "figure" or "table"
    """
    if kind == "figure":
        # If obj is a matplotlib Figure, save directly
        if hasattr(obj, "savefig"):
            obj.savefig(f"outputs/figures/{name}.png", dpi=300, bbox_inches="tight")
        else:
            # fallback: save current plt figure
            plt.savefig(f"outputs/figures/{name}.png", dpi=300, bbox_inches="tight")
        plt.close()

    elif kind == "table":
        # Assume it's a pandas DataFrame
        obj.to_csv(f"outputs/tables/{name}.csv", index=False)

    else:
        raise ValueError("kind must be 'figure' or 'table'")

    print(f"✅ Saved {kind}: {name}")

In [2]:
boston_col_head = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
'''
    CRIM - per capita crime rate by town
    ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
    INDUS - proportion of non-retail business acres per town.
    CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
    NOX - nitric oxides concentration (parts per 10 million)
    RM - average number of rooms per dwelling
    AGE - proportion of owner-occupied units built prior to 1940
    DIS - weighted distances to five Boston employment centres
    RAD - index of accessibility to radial highways
    TAX - full-value property-tax rate per $10,000
    PTRATIO - pupil-teacher ratio by town
    B - 1000(Bk - 0.63)^2 where Bk is the proportion of black people by town
    LSTAT - % lower status of the population
    MEDV - Median value of owner-occupied homes in $1000's
'''
#run this then can use boston correctly

"\n    CRIM - per capita crime rate by town\n    ZN - proportion of residential land zoned for lots over 25,000 sq.ft.\n    INDUS - proportion of non-retail business acres per town.\n    CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)\n    NOX - nitric oxides concentration (parts per 10 million)\n    RM - average number of rooms per dwelling\n    AGE - proportion of owner-occupied units built prior to 1940\n    DIS - weighted distances to five Boston employment centres\n    RAD - index of accessibility to radial highways\n    TAX - full-value property-tax rate per $10,000\n    PTRATIO - pupil-teacher ratio by town\n    B - 1000(Bk - 0.63)^2 where Bk is the proportion of black people by town\n    LSTAT - % lower status of the population\n    MEDV - Median value of owner-occupied homes in $1000's\n"

In [3]:
boston = pd.read_csv('Boston.csv', delim_whitespace=True, header=None)
boston.columns = boston_col_head
boston.head()
#

/tmp/ipython-input-3253699271.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  boston = pd.read_csv('Boston.csv', delim_whitespace=True, header=None)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [4]:
boston.dtypes

,0
CRIM,float64
ZN,float64
INDUS,float64
CHAS,int64
NOX,float64
RM,float64
AGE,float64
DIS,float64
RAD,int64
TAX,float64


We find there are 506 records each representing a town or suburb in the Boston area.  Some quantitative columns are aggregated over houses.  E.g. MEDV is the median home value, RM is the average number of rooms for each house.  Some quantitative columns are rates not necessarily indicated by house.  E.g. ZN is the a percentage of land in the area, B is a statistic based on the number of black people living in the area.

CHAS is an indicator for surrounding the Charles River.  RAD is an index for accessibility to radial highways, though there are many values concentrated at 24.  So, these variables might be treated as qualitative.  Moreover, although TAX has viable interpretation as a quantitative variable, the fact that it clusters around tracts might require it to be treated categorically.  Random Forest regressors will be very important to accommodating the undelrying structure of the data.

# Section 1 (EDA)

In [5]:
boston.dtypes
num_records = len(boston)
print(f"Number of records: {num_records}")

Number of records: 506


In [6]:
specvar1 = 'RAD'
specvar2 = 'TAX'
specvar3 = 'B'
figs, ax = plt.subplots(nrows=2, ncols=3, figsize=(10, 10))
sns.scatterplot(data=boston, x='CRIM', y='LSTAT', ax=ax[0,0])
sns.scatterplot(data=boston, x='CRIM', y='RAD', ax=ax[0,1])
sns.scatterplot(data=boston, x='CRIM', y='TAX', ax=ax[1,0])
sns.scatterplot(data=boston, x='CRIM', y = 'B', ax=ax[1,1])
sns.scatterplot(data=boston, x=specvar1, y=specvar2, ax=ax[0,2])
sns.scatterplot(data=boston, x=specvar1, y=specvar3, ax=ax[1,2])

ax[0,0].set_title("CRIM vs LSTAT")
ax[0,1].set_title("CRIM vs RAD")
ax[1,0].set_title("CRIM vs TAX")
ax[1,1].set_title("CRIM vs B")
ax[0,2].set_title(f"{specvar1} vs {specvar2}")
ax[1,2].set_title(f"{specvar1} vs {specvar3}")

plt.tight_layout() # Adjust layout to prevent titles overlapping

save_output(figs, "boston_scatter_figs_all")

cols_restricted = ['CRIM', 'LSTAT', 'RAD', 'TAX', 'B']
data_restricted = boston[cols_restricted]
corr_restricted = data_restricted.corr()
print(corr_restricted)

✅ Saved figure: boston_scatter_figs_all
           CRIM     LSTAT       RAD       TAX         B
CRIM   1.000000  0.455621  0.625505  0.582764 -0.385064
LSTAT  0.455621  1.000000  0.488676  0.543993 -0.366087
RAD    0.625505  0.488676  1.000000  0.910228 -0.444413
TAX    0.582764  0.543993  0.910228  1.000000 -0.441808
B     -0.385064 -0.366087 -0.444413 -0.441808  1.000000


In [7]:
rad_24_indicator = boston['RAD'].apply(lambda x: 1 if x == 24 else 0)
boston['RAD_24'] = rad_24_indicator

In [8]:
corr_matrix = boston.corr()
corr_matrix

mask = np.tril(np.ones_like(corr_matrix, dtype=bool))

# Apply the mask: set lower triangle to NaN
upper_tri = corr_matrix.mask(mask)
#print(upper_tri.round(2))

fig, ax = plt.subplots(figsize=(8,6))

# Plot heatmap on this figure/axes
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f",
            cmap="coolwarm", annot_kws={"size": 8}, ax=ax)

ax.set_title("Upper-Triangle Correlation Matrix")

save_output(fig, "corr_matrix")

✅ Saved figure: corr_matrix


Given the scatter plots and the correlation matrix, one can compare the linear relationship between pairs of variables without worrying about a potential non-linear relationship.  From the heatmap, the factors most correlated with crime rate are RAD, TAX, and LSTAT.

However, it should be noted that most of the variation in crime occurs in highly specific brackets of RAD and TAX.  The crime rate is near 0 across all other brackets.  Note that RAD and TAX are very highly correlated.

We create an indicator variable RAD_24 that equals 1 when RAD equals 24 and equals 0 otherwise.

In [9]:
scaler = StandardScaler()
boston_scaled = scaler.fit_transform(boston)
boston_scaled = pd.DataFrame(boston_scaled, columns=boston.columns)
boston_scaled.head()
#save_output(boston_scaled_scatter, "boston_scaled_scatter")

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,RAD_24
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562,0.159686,-0.594089
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439,-0.101524,-0.594089
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727,1.324247,-0.594089
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517,1.182758,-0.594089
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501,1.487503,-0.594089


In [10]:
boston.describe()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,RAD_24
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806,0.260870
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104,0.439543
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000,0.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000,0.000000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000,0.000000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000,1.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000,1.000000


Range of predictors indicates differing scales.  Crime ``rate'' varies from 0 to 88.98.  Tax rates vary from 187 to 711.  Pupil-teacher ratios vary from 12.6 to 22.  Many of these rates are clearly raw percentages, but it is not clear at all what the format of the tax rate is.

In [11]:
chas_1count = boston[boston['CHAS']==1].shape[0]
chas_0count = boston[boston['CHAS']==0].shape[0]
print(f"Number of records with CHAS=1: {chas_1count}")
print(f"Number of records with CHAS=0: {chas_0count}")

ptratio_median = boston['PTRATIO'].median()
print(f"Median of PTRATIO: {ptratio_median}")

min_medv = boston[boston['MEDV']==boston['MEDV'].min()] #we call boston['MED'] twice here because we do not necessarily know the min
max_medv = boston[boston['MEDV']==boston['MEDV'].max()]
print(min_medv)

Number of records with CHAS=1: 35
Number of records with CHAS=0: 471
Median of PTRATIO: 19.05
        CRIM   ZN  INDUS  CHAS    NOX     RM    AGE     DIS  RAD    TAX  \
398  38.3518  0.0   18.1     0  0.693  5.453  100.0  1.4896   24  666.0   
405  67.9208  0.0   18.1     0  0.693  5.683  100.0  1.4254   24  666.0   

     PTRATIO       B  LSTAT  MEDV  RAD_24  
398     20.2  396.90  30.59   5.0       1  
405     20.2  384.97  22.98   5.0       1  


We compute a few statistics involving the qualitative CHAS and quantitative PTRATIO and MEDV.  

There are 35 suburbs that bound the Charles river, indicated by CHAS being equal to 1.

The median pupil-teacher ratio is 19.05.

There are two Boston suburbs with the minimum median value (MEDV), indicated by 398 and 405.  They have highly similar statistics except for CRIM and LSTAT.

In [12]:
greater_than_7 = boston[boston['RM'] > 7]
greater_than_8 = boston[boston['RM'] > 8]
print(greater_than_7,len(greater_than_7))
print(greater_than_8,len(greater_than_8))

         CRIM    ZN  INDUS  CHAS     NOX     RM   AGE     DIS  RAD    TAX  \
2     0.02729   0.0   7.07     0  0.4690  7.185  61.1  4.9671    2  242.0   
4     0.06905   0.0   2.18     0  0.4580  7.147  54.2  6.0622    3  222.0   
40    0.03359  75.0   2.95     0  0.4280  7.024  15.8  5.4011    3  252.0   
55    0.01311  90.0   1.22     0  0.4030  7.249  21.9  8.6966    5  226.0   
64    0.01951  17.5   1.38     0  0.4161  7.104  59.5  9.2229    3  216.0   
..        ...   ...    ...   ...     ...    ...   ...     ...  ...    ...   
364   3.47428   0.0  18.10     1  0.7180  8.780  82.9  1.9047   24  666.0   
370   6.53876   0.0  18.10     1  0.6310  7.016  97.5  1.2024   24  666.0   
375  19.60910   0.0  18.10     0  0.6710  7.313  97.9  1.3163   24  666.0   
453   8.24809   0.0  18.10     0  0.7130  7.393  99.3  2.4527   24  666.0   
482   5.73116   0.0  18.10     0  0.5320  7.061  77.0  3.4106   24  666.0   

     PTRATIO       B  LSTAT  MEDV  RAD_24  
2       17.8  392.83   4.03  34

In [13]:
#rad_plot = sns.histplot(boston['B'])
#save_output(rad_plot, "rad_plot")
#boston_scatter = sns.pairplot(boston)
data7 = boston[boston['RM'] > 7]
data_in = data7

#specvar1 = 'RAD' #by default
#specvar2 = 'TAX' #by default
#specvar3 = 'B' #by default


figs, ax = plt.subplots(nrows=2, ncols=3, figsize=(10, 10))
sns.scatterplot(data=data_in, x='CRIM', y='LSTAT', ax=ax[0,0])
sns.scatterplot(data=data_in, x='CRIM', y='RAD', ax=ax[0,1])
sns.scatterplot(data=data_in, x='CRIM', y='TAX', ax=ax[1,0])
sns.scatterplot(data=data_in, x='CRIM', y = 'B', ax=ax[1,1])
sns.scatterplot(data=data_in, x=specvar1, y=specvar2, ax=ax[0,2])
sns.scatterplot(data=data_in, x=specvar1, y=specvar3, ax=ax[1,2])



ax[0,0].set_title("CRIM vs LSTAT")
ax[0,1].set_title("CRIM vs RAD")
ax[1,0].set_title("CRIM vs TAX")
ax[1,1].set_title("CRIM vs B")
ax[0,2].set_title(f"{specvar1} vs {specvar2}")
ax[1,2].set_title(f"{specvar1} vs {specvar3}")

save_output(figs, "boston_scatter_figs_RM>7")

plt.tight_layout() # Adjust layout to prevent titles overlapping




✅ Saved figure: boston_scatter_figs_RM>7


<Figure size 640x480 with 0 Axes>

In [14]:
#rad_plot = sns.histplot(boston['B'])
#save_output(rad_plot, "rad_plot")
#boston_scatter = sns.pairplot(boston)
data8 = boston[boston['RM'] > 8]
data_in = data8

#specvar1 = 'RAD' #by default
#specvar2 = 'TAX' #by default
#specvar3 = 'B' #by default


figs, ax = plt.subplots(nrows=2, ncols=3, figsize=(10, 10))
sns.scatterplot(data=data_in, x='CRIM', y='LSTAT', ax=ax[0,0])
sns.scatterplot(data=data_in, x='CRIM', y='RAD', ax=ax[0,1])
sns.scatterplot(data=data_in, x='CRIM', y='TAX', ax=ax[1,0])
sns.scatterplot(data=data_in, x='CRIM', y = 'B', ax=ax[1,1])
sns.scatterplot(data=data_in, x=specvar1, y=specvar2, ax=ax[0,2])
sns.scatterplot(data=data_in, x=specvar1, y=specvar3, ax=ax[1,2])



ax[0,0].set_title("CRIM vs LSTAT")
ax[0,1].set_title("CRIM vs RAD")
ax[1,0].set_title("CRIM vs TAX")
ax[1,1].set_title("CRIM vs B")
ax[0,2].set_title(f"{specvar1} vs {specvar2}")
ax[1,2].set_title(f"{specvar1} vs {specvar3}")

save_output(figs, "boston_scatter_figs_RM>8")

plt.tight_layout() # Adjust layout to prevent titles overlapping



✅ Saved figure: boston_scatter_figs_RM>8


<Figure size 640x480 with 0 Axes>

In [15]:
#rad_plot = sns.histplot(boston['B'])
#save_output(rad_plot, "rad_plot")
#boston_scatter = sns.pairplot(boston)
boston_rad_24 = boston[boston['RAD']==24]
data_in = boston_rad_24

#specvar1 = 'RAD' #by default
#specvar2 = 'TAX' #by default
#specvar3 = 'B' #by default


figs, ax = plt.subplots(nrows=2, ncols=3, figsize=(10, 10))
sns.scatterplot(data=data_in, x='CRIM', y='LSTAT', ax=ax[0,0])
sns.scatterplot(data=data_in, x='CRIM', y='RAD', ax=ax[0,1])
sns.scatterplot(data=data_in, x='CRIM', y='TAX', ax=ax[1,0])
sns.scatterplot(data=data_in, x='CRIM', y = 'B', ax=ax[1,1])
sns.scatterplot(data=data_in, x=specvar1, y=specvar2, ax=ax[0,2])
sns.scatterplot(data=data_in, x=specvar1, y=specvar3, ax=ax[1,2])



ax[0,0].set_title("CRIM vs LSTAT")
ax[0,1].set_title("CRIM vs RAD")
ax[1,0].set_title("CRIM vs TAX")
ax[1,1].set_title("CRIM vs B")
ax[0,2].set_title(f"{specvar1} vs {specvar2}")
ax[1,2].set_title(f"{specvar1} vs {specvar3}")

save_output(figs, "boston_scatter_figs_RAD=24")

plt.tight_layout() # Adjust layout to prevent titles overlapping

✅ Saved figure: boston_scatter_figs_RAD=24


<Figure size 640x480 with 0 Axes>

In [16]:
#rad_plot = sns.histplot(boston['B'])
#save_output(rad_plot, "rad_plot")
#boston_scatter = sns.pairplot(boston)
boston_not_rad_24 = boston[boston['RAD']!=24]
data_in = boston_not_rad_24

#specvar1 = 'RAD' #by default
#specvar2 = 'TAX' #by default
#specvar3 = 'B' #by default


figs, ax = plt.subplots(nrows=2, ncols=3, figsize=(10, 10))
sns.scatterplot(data=data_in, x='CRIM', y='LSTAT', ax=ax[0,0])
sns.scatterplot(data=data_in, x='CRIM', y='RAD', ax=ax[0,1])
sns.scatterplot(data=data_in, x='CRIM', y='TAX', ax=ax[1,0])
sns.scatterplot(data=data_in, x='CRIM', y = 'B', ax=ax[1,1])
sns.scatterplot(data=data_in, x=specvar1, y=specvar2, ax=ax[0,2])
sns.scatterplot(data=data_in, x=specvar1, y=specvar3, ax=ax[1,2])



ax[0,0].set_title("CRIM vs LSTAT")
ax[0,1].set_title("CRIM vs RAD")
ax[1,0].set_title("CRIM vs TAX")
ax[1,1].set_title("CRIM vs B")
ax[0,2].set_title(f"{specvar1} vs {specvar2}")
ax[1,2].set_title(f"{specvar1} vs {specvar3}")

save_output(figs, "boston_scatter_figs_RAD~=24")

plt.tight_layout() # Adjust layout to prevent titles overlapping

✅ Saved figure: boston_scatter_figs_RAD~=24


<Figure size 640x480 with 0 Axes>

In [17]:
cols_restricted = ['CRIM', 'LSTAT', 'RAD', 'RAD_24','TAX', 'B']
data_restricted = data7[cols_restricted]
corr_restricted = data_restricted.corr()
#print(corr_restricted)


corr_matrix = corr_restricted

mask = np.tril(np.ones_like(corr_matrix, dtype=bool))

# Apply the mask: set lower triangle to NaN
upper_tri = corr_matrix.mask(mask)
#print(upper_tri.round(2))

fig, ax = plt.subplots(figsize=(8,6))

# Plot heatmap on this figure/axes
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f",
            cmap="coolwarm", annot_kws={"size": 8}, ax=ax)

ax.set_title("Upper-Triangle Correlation Matrix")
save_output(fig, "corr_matrix_RAD_24_RM>7")

✅ Saved figure: corr_matrix_RAD_24_RM>7


In [18]:
cols_restricted = ['CRIM', 'LSTAT', 'RAD', 'RAD_24','TAX', 'B']
data_restricted = data8[cols_restricted]
corr_restricted = data_restricted.corr()
#print(corr_restricted)

corr_matrix = corr_restricted

mask = np.tril(np.ones_like(corr_matrix, dtype=bool))

# Apply the mask: set lower triangle to NaN
upper_tri = corr_matrix.mask(mask)
#print(upper_tri.round(2))

fig, ax = plt.subplots(figsize=(8,6))

# Plot heatmap on this figure/axes
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f",
            cmap="coolwarm", annot_kws={"size": 8}, ax=ax)

ax.set_title("Upper-Triangle Correlation Matrix")

save_output(fig, "corr_matrix_RAD_24_RM>8")

✅ Saved figure: corr_matrix_RAD_24_RM>8


In [19]:
cols_restricted = ['CRIM', 'LSTAT', 'RAD', 'RAD_24','TAX', 'B']
data_restricted = boston_rad_24[cols_restricted]
corr_restricted = data_restricted.corr()
#print(corr_restricted)


corr_matrix = corr_restricted

mask = np.tril(np.ones_like(corr_matrix, dtype=bool))

# Apply the mask: set lower triangle to NaN
upper_tri = corr_matrix.mask(mask)
#print(upper_tri.round(2))

fig, ax = plt.subplots(figsize=(8,6))

# Plot heatmap on this figure/axes
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f",
            cmap="coolwarm", annot_kws={"size": 8}, ax=ax)

ax.set_title("Upper-Triangle Correlation Matrix")
save_output(fig, "corr_matrix_RAD=24")

✅ Saved figure: corr_matrix_RAD=24


In [20]:
cols_restricted = ['CRIM', 'LSTAT', 'RAD', 'RAD_24','TAX', 'B']
data_restricted = boston_not_rad_24[cols_restricted]
corr_restricted = data_restricted.corr()
#print(corr_restricted)


corr_matrix = corr_restricted

mask = np.tril(np.ones_like(corr_matrix, dtype=bool))

# Apply the mask: set lower triangle to NaN
upper_tri = corr_matrix.mask(mask)
#print(upper_tri.round(2))

fig, ax = plt.subplots(figsize=(8,6))

# Plot heatmap on this figure/axes
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f",
            cmap="coolwarm", annot_kws={"size": 8}, ax=ax)

ax.set_title("Upper-Triangle Correlation Matrix")
save_output(fig, "corr_matrix_RAD~=24")

✅ Saved figure: corr_matrix_RAD~=24


There are 64 dwellings with more than 7 rooms and 13 with more than 8 rooms.  Looking at the scatter plots and correlation matrices restricted to these entries, it appears there is more decisive correlation between CRIM and other variables.  This might suggest that features might be more predictive when restricted to parts of the dataset.

# Section 2 (Restricting based on RAD)

In [21]:
boston_rad_24 = boston[boston['RAD']==24]
boston_non_rad_24 = boston[boston['RAD']!=24]
print(boston_rad_24.shape)
print(boston_non_rad_24.shape)

(132, 15)
(374, 15)


In [22]:
scaler = StandardScaler()
boston_rad_24_scaled = scaler.fit_transform(boston_rad_24)
boston_rad_24_scaled = pd.DataFrame(boston_rad_24_scaled, columns=boston_rad_24.columns)
boston_non_rad_24_scaled = scaler.fit_transform(boston_non_rad_24)
boston_non_rad_24_scaled = pd.DataFrame(boston_non_rad_24_scaled, columns=boston_non_rad_24.columns)

In [23]:
y1 = boston_rad_24_scaled['CRIM']
X1 = boston_rad_24_scaled.drop(columns = ['CRIM', 'RAD_24'])

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

model = LinearRegression()
resultsErr = []

model.fit(X1_train, y1_train)
coef_24 = pd.DataFrame({'Feature': X1.columns, 'Coefficient': model.coef_})
resultsErr.append({
    'Model': 'RAD = 24',
    'R2': model.score(X1_test, y1_test),
    'RMSE': mean_squared_error(y1_test, model.predict(X1_test)),
})
Importances = pd.DataFrame({'Feature': X1.columns, 'RAD = 24': model.coef_})
#print(resultsErr)


In [24]:
y2 = boston_non_rad_24_scaled['CRIM']
X2 = boston_non_rad_24_scaled.drop(columns = ['CRIM', 'RAD_24'])

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X2_train, y2_train)
coef_non_24 = pd.DataFrame({'Feature': X2.columns, 'Coefficient': model.coef_})
resultsErr.append({
    'Model': 'RAD < 24',
    'R2': model.score(X2_test, y2_test),
    'RMSE': mean_squared_error(y2_test, model.predict(X2_test)),
})
tempImportances = pd.DataFrame({'Feature': X2.columns, 'RAD < 24': model.coef_})
Importances = pd.merge(Importances, tempImportances, on='Feature', how='outer')
#print(resultsErr)


In [25]:
y2 = boston_rad_24_scaled['CRIM']
X2 = boston_rad_24_scaled.drop(columns = ['CRIM', 'RAD_24'])

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

model = Lasso(alpha = 0.1)

model.fit(X2_train, y2_train)
coef_non_24 = pd.DataFrame({'Feature': X2.columns, 'Coefficient': model.coef_})
resultsErr.append({
    'Model': 'Lasso, RAD = 24',
    'R2': model.score(X2_test, y2_test),
    'RMSE': mean_squared_error(y2_test, model.predict(X2_test)),
})
tempImportances = pd.DataFrame({'Feature': X2.columns, 'Lasso, RAD = 24': model.coef_})
Importances = pd.merge(Importances, tempImportances, on='Feature', how='outer')
#print(resultsErr)

In [26]:
y2 = boston_non_rad_24_scaled['CRIM']
X2 = boston_non_rad_24_scaled.drop(columns = ['CRIM', 'RAD_24'])

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

model = Lasso(alpha = 0.1)

model.fit(X2_train, y2_train)
coef_non_24 = pd.DataFrame({'Feature': X2.columns, 'Coefficient': model.coef_})
resultsErr.append({
    'Model': 'Lasso, RAD < 24',
    'R2': model.score(X2_test, y2_test),
    'RMSE': mean_squared_error(y2_test, model.predict(X2_test)),
})
tempImportances = pd.DataFrame({'Feature': X2.columns, 'Lasso, RAD < 24': model.coef_})
Importances = pd.merge(Importances, tempImportances, on='Feature', how='outer')
#print(resultsErr)

In [27]:
metrics_df = pd.DataFrame(resultsErr)
metrics_df2 = pd.DataFrame(Importances)

from IPython.display import display
display(metrics_df)
display(metrics_df2)

tables = {
    'metrics_df': metrics_df,
    'metrics_df2': metrics_df2,
}

#metrics_df.to_csv('OLSErr.csv', index=False)
#metrics_df2.to_csv('OLSImportances.csv', index=False)

save_output(metrics_df, "Split RAD Error", kind = "table")
save_output(metrics_df2, "Split RAD Importances", kind = "table")

,Model,R2,RMSE
0,RAD = 24,0.225724,0.920142
1,RAD < 24,0.705292,0.248964
2,"Lasso, RAD = 24",0.218697,0.928492
3,"Lasso, RAD < 24",0.741799,0.218123


,Feature,RAD = 24,RAD < 24,"Lasso, RAD = 24","Lasso, RAD < 24"
0,AGE,-7.415527e-02,0.086699,-0.000000,0.000000
1,B,3.263584e-02,-0.217375,-0.000000,-0.139769
2,CHAS,7.432759e-03,0.047477,-0.000000,0.000000
3,DIS,-5.590578e-01,0.228571,-0.303985,-0.000000
4,INDUS,-1.665335e-16,0.111678,0.000000,0.000000
5,LSTAT,-2.145231e-01,0.001717,0.000000,0.000000
6,MEDV,-4.795345e-01,0.244615,-0.204832,-0.000000
7,NOX,-1.989192e-01,0.679055,-0.031796,0.601572
8,PTRATIO,0.000000e+00,-0.124736,0.000000,-0.052908
9,RAD,2.775558e-17,0.034039,0.000000,0.000000


✅ Saved table: Split RAD Error
✅ Saved table: Split RAD Importances


The RAD = 24 set is less heterogeneous, so it will be harder to explain the variation in the prediction from the features that we have.  Thus, R^2 and RMSE will show a better performance in predicting the crime rate in the RAD < 24 set.

In implementing Lasso with alpha = 0.1 on the RAD < 24 set, the R^2 increased slightly while RMSE decreased slightly.  This means that Lasso is reducing the effect of redundant features though it makes a simpler model.  On the other hand, when applied to the RAD = 24 set, the R^2 decreased while RMSE increased.  There is almost no signal to find, so shrinkage adds bias without meaningful reduction in variance.

In [28]:
#treat all RAD as an ordinal amount but retain the RAD_24 and CHAS as binary.  Then the data will work in the Random Forest regressor in sklearn
X = boston_scaled.drop(columns = ['CRIM'])
y = boston_scaled['CRIM']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




model = RandomForestRegressor(n_estimators=100, max_depth = None, random_state=42)
resultsErr2 = []
model.fit(X_train,y_train)
coef_rf_reg = pd.DataFrame({'Feature': X.columns, 'Random Forest-Reg': model.feature_importances_})

resultsErr2.append({
    'Model': 'Random Forest-Reg',
    'R2': model.score(X_test, y_test),
    'RMSE': mean_squared_error(y_test, model.predict(X_test)),
})


In [29]:
#treat all RAD as binary (through RAD_24) and CHAS as binary.  Then the data will work in the Random Forest regressor in sklearn
X = boston_scaled.drop(columns = ['CRIM','RAD'])
y = boston_scaled['CRIM']





X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, max_depth = None, random_state=42)
model.fit(X_train,y_train)
coef_rf_bin = pd.DataFrame({'Feature': X.columns, 'Random Forest-BinRAD': model.feature_importances_})

resultsErr2.append({
    'Model': 'Random Forest-BinRAD',
    'R2': model.score(X_test, y_test),
    'RMSE': mean_squared_error(y_test, model.predict(X_test)),
})

coef_rf_reg = pd.merge(coef_rf_reg, coef_rf_bin, on='Feature', how='outer')

In [30]:
metrics_df = pd.DataFrame(resultsErr2)
metrics_df2 = pd.DataFrame(coef_rf_reg)

from IPython.display import display
display(metrics_df)
display(metrics_df2)

tables = {
    'metrics_df': metrics_df,
    'metrics_df2': metrics_df2,
}

save_output(metrics_df, "RF Error", kind = "table")
save_output(metrics_df2, "RF Importances", kind = "table")

,Model,R2,RMSE
0,Random Forest-Reg,0.764872,0.175454
1,Random Forest-BinRAD,0.753004,0.184310


,Feature,Random Forest-Reg,Random Forest-BinRAD
0,AGE,0.032786,0.028169
1,B,0.091398,0.090858
2,CHAS,0.000298,0.000304
3,DIS,0.117459,0.125794
4,INDUS,0.000794,0.000727
5,LSTAT,0.046903,0.047730
6,MEDV,0.306398,0.303020
7,NOX,0.015504,0.016751
8,PTRATIO,0.001352,0.001488
9,RAD,0.095516,NaN


✅ Saved table: RF Error
✅ Saved table: RF Importances


The random forest in sklearn can handle binary indicators, but it is important to see if there is a role in the specific values in RAD.  So, we perform a random forest model in two cases.  The first case is where RAD is removed but leave only an indicator for RAD = 24.  The second case is where the (normalized) values of RAD are retained.  Both models perform very well in terms of R^2 and RMSE; they explain a similar amount of variance to Lasso for RAD < 24 but even have a significant drop in test error as shown with RMSE.  The features selected in each are basically the same.  MEDV and DIS are important features, but the representative for RAD (whether RAD or RAD_24) shows a similar degree of importance.  This shows that specific values of RAD are not conveying much more information compared to whether RAD is simply equal to 24.  At this point, it seems simpler and more consistent to use the RAD_24 indicator.